# Batch correction

## Motivation

As could be seen for our earlier visualized ADT data, batch effects between samples are very pronounced.
Hence, batch correction to mitigate this effect is required.
Generally, no purpose build methods for the batch correction of ADT data have been developed, and we therefore suggest applying methods designed for transcriptomics data to ADT data.

## Environment setup

In [1]:
import scanpy as sc
import muon as mu
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import harmonypy as hm
import warnings

warnings.simplefilter(action="ignore", category=UserWarning)

/home/icb/ciro.suastegui/miniconda3/envs/citeseq_pp/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading the data

In [ ]:
raw = mu.read("path")  ## TODO
filtered = mu.read("")  ## TODO

It is not yet clear which batch effect correction works best for ADT data. For general purposes we recommend scVI {cite}`Lopez2018` or Harmony {cite}`Korsunsky2019` to perform batch correction of the data due to their robust performance on scRNA-seq data.

## Harmony

In [ ]:
%%time
ho = hm.run_harmony(filtered["prot"].X, filtered["prot"].obs, ["donor"])

In [ ]:
pc_std = np.std(ho.Z_corr, axis=1).tolist()

In [ ]:
sns.scatterplot(x=range(0, len(pc_std)), y=sorted(pc_std, reverse=True))

In [ ]:
filtered["prot"].obsm["X_pcahm"] = ho.Z_corr.transpose()

In [ ]:
filtered["prot"].obsm

In [ ]:
%%time
sc.pp.neighbors(filtered["prot"], n_pcs=30, use_rep="X_pcahm")
sc.tl.umap(filtered["prot"])

In [ ]:
sc.tl.tsne(filtered["prot"])

In [ ]:
sc.pl.umap(filtered["prot"], color=["donor", "batch"])

As we can see here, the cells of different donors are much more intermixed in the embedding than before.

In [ ]:
sc.pl.umap(filtered["prot"], color=["CD4-1", "CD8", "CD3"])
sc.pl.umap(filtered["prot"], color=["CD14-1", "CD16"])

We check the expression of a few marker genes to confirm that separate cell types are still separate from each other. We can see that T-Cells still form a separate population that is further split into CD4 and CD8 T-Cells.

In following steps, you can now go ahead and cluster and annotate the cells in a similar process as it is described in the annotation chapter[LINK].
Here, we have only used the ADT part of the data and thus lost all of the information contained in the RNA part of the study.
In other chapters, we will explore how you can make use of both modalities jointly, which allows for a more detailed cell type annotation for example.

## Key takeaways

TODO

## References

```{bibliography}
:filter: docname in docnames
```